In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()
df = df.drop(columns=["home_ownership","verification_status","issue_d", "pymnt_plan","initial_list_status","next_pymnt_d","application_type","hardship_flag","debt_settlement_flag"])


# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,35000.0,0.1308,797.80,125890.0,Fully Paid,30.48,0.0,0.0,14.0,0.0,...,0.0,2.0,100.0,80.0,0.0,0.0,399376.0,131350.0,32700.0,130876.0
1,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0.0,3.0,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0
2,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,0.0,6.0,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0
3,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,0.0,4.0,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0
4,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0.0,1.0,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0


# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.drop(columns="loan_status")


# Create our target
y = df["loan_status"]

In [7]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,70423.000000,70423.000000,70423.000000,7.042300e+04,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,...,70423.000000,70423.000000,70423.000000,70423.000000,70423.000000,70423.0,7.042300e+04,7.042300e+04,70423.000000,7.042300e+04
mean,16621.372989,0.128063,479.478688,8.843486e+04,21.745274,0.217244,0.500632,12.583787,0.125939,17549.650966,...,0.052213,2.231657,95.073013,30.515630,0.125882,0.0,2.100903e+05,6.131966e+04,29748.067038,5.572026e+04
std,10282.268052,0.048339,288.457221,1.218578e+05,20.429814,0.718822,0.760803,6.028804,0.336711,21880.270487,...,0.393946,1.907077,8.311660,33.618896,0.336648,0.0,1.928094e+05,5.735407e+04,26888.846929,5.090730e+04
min,1000.000000,0.060000,30.640000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,264.930000,5.000000e+04,13.840000,0.000000,0.000000,8.000000,0.000000,6240.500000,...,0.000000,1.000000,93.100000,0.000000,0.000000,0.0,6.700800e+04,2.645200e+04,11600.000000,2.286200e+04
50%,15000.000000,0.118000,402.890000,7.300000e+04,19.710000,0.000000,0.000000,11.000000,0.000000,12014.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.469600e+05,4.533800e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,646.740000,1.040000e+05,26.620000,0.000000,1.000000,16.000000,0.000000,21657.500000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036355e+05,7.656750e+04,39300.000000,7.250600e+04
max,40000.000000,0.308400,1676.230000,9.000000e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [8]:
# Check the balance of our target values
y.value_counts()

low_risk       68470
Fully Paid      1584
high_risk        347
Charged Off       22
Name: loan_status, dtype: int64

In [9]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(52817, 76)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [10]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_train, y_train)
rf_model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=78, verbose=0,
                       warm_start=False)

In [11]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score
y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8182498962576196

In [12]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[    5,     0,     0,     0],
       [    0,   396,     0,     0],
       [    0,     0,    24,    63],
       [    0,    37,    12, 17069]], dtype=int64)

In [13]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       1.00      1.00      1.00      1.00      1.00      1.00         5
 Fully Paid       0.91      1.00      1.00      0.96      1.00      1.00       396
  high_risk       0.67      0.28      1.00      0.39      0.53      0.26        87
   low_risk       1.00      1.00      0.87      1.00      0.93      0.88     17118

avg / total       0.99      0.99      0.87      0.99      0.93      0.88     17606



In [14]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.20469625430304716, 'out_prncp_inv'),
 (0.1669014783927051, 'out_prncp'),
 (0.13790965475269373, 'last_pymnt_amnt'),
 (0.10526097373582442, 'total_pymnt'),
 (0.08866180195828663, 'total_rec_prncp'),
 (0.08654155714932944, 'total_pymnt_inv'),
 (0.03525406386194753, 'total_rec_int'),
 (0.006769002298293533, 'installment'),
 (0.006353897276327656, 'loan_amnt'),
 (0.005048816431097407, 'dti'),
 (0.004823475645865944, 'mo_sin_old_rev_tl_op'),
 (0.0045261967123492895, 'revol_bal'),
 (0.004289207983743566, 'annual_inc'),
 (0.0042762607737164855, 'max_bal_bc'),
 (0.0042253780916923405, 'bc_util'),
 (0.004202048875030067, 'total_bal_ex_mort'),
 (0.004171187761550381, 'avg_cur_bal'),
 (0.004119145073098547, 'tot_hi_cred_lim'),
 (0.004109014374810499, 'bc_open_to_buy'),
 (0.004102005294206979, 'total_bal_il'),
 (0.00406315588826354, 'mo_sin_old_il_acct'),
 (0.004007627844979317, 'tot_cur_bal'),
 (0.003985234698414017, 'total_il_high_credit_limit'),
 (0.003899900882879271, 'int_rate'),
 (0.0038

### Easy Ensemble AdaBoost Classifier

In [15]:
# Train the Classifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier
rf_model = EasyEnsembleClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_train, y_train)
rf_model

EasyEnsembleClassifier(base_estimator=None, n_estimators=500, n_jobs=1,
                       random_state=78, replacement=False,
                       sampling_strategy='auto', verbose=0, warm_start=False)

In [16]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score
y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8152567271394096

In [17]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[    5,     0,     0,     0],
       [   33,   363,     0,     0],
       [    0,     0,    59,    28],
       [    6,    41,  5667, 11404]], dtype=int64)

In [19]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

Charged Off       0.11      1.00      1.00      0.20      1.00      1.00         5
 Fully Paid       0.90      0.92      1.00      0.91      0.96      0.91       396
  high_risk       0.01      0.68      0.68      0.02      0.68      0.46        87
   low_risk       1.00      0.67      0.94      0.80      0.79      0.61     17118

avg / total       0.99      0.67      0.94      0.80      0.80      0.62     17606

